In [1]:
import os

In [2]:
%pwd

'd:\\Git-Hub Projects\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Git-Hub Projects\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Chicken_disease_classifier.constants import *
from Chicken_disease_classifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config =  DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from Chicken_disease_classifier import logger
from Chicken_disease_classifier.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} with following info: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        This function Extracts the zip file into the data directory
        Args:
            zip_file_path: str
        Returns: None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-16 21:35:13,019] {D:\Git-Hub Projects\Chicken-Disease-Classification\src\Chicken_disease_classifier\utils\common.py:31} INFO - Successfully read the yaml file from config\config.yaml
[2025-02-16 21:35:13,021] {D:\Git-Hub Projects\Chicken-Disease-Classification\src\Chicken_disease_classifier\utils\common.py:31} INFO - Successfully read the yaml file from params.yaml
[2025-02-16 21:35:13,022] {D:\Git-Hub Projects\Chicken-Disease-Classification\src\Chicken_disease_classifier\utils\common.py:51} INFO - Directory created at : artifacts
[2025-02-16 21:35:13,022] {D:\Git-Hub Projects\Chicken-Disease-Classification\src\Chicken_disease_classifier\utils\common.py:51} INFO - Directory created at : artifacts/data_ingestion
[2025-02-16 21:35:30,756] {C:\Users\Kabyik\AppData\Local\Temp\ipykernel_18780\1252037403.py:11} INFO - Downloaded file: artifacts/data_ingestion/data.zip with following info: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: 